In [1]:
# Imports

from src.resources import Generator, Load, Storage, Vehicle, BinaryResource
from src.parsers import HMParser, PROCSIMParser
from src.scenes import HMProblemScene

import numpy as np

In [2]:
# PROCSIM data

procsim_data = PROCSIMParser('data/procsim/netload.csv')
procsim_data.parse()

# Resample the data to match the HM format
gen_resampled = procsim_data.generator.resample('1H').mean()
gen_resampled = np.reshape(gen_resampled, (1, len(gen_resampled)))

load_resampled = procsim_data.load.resample('1H').mean()
load_resampled = np.reshape(load_resampled, (1, len(load_resampled)))

In [3]:
# HM data to complete the problem formulation

data = HMParser('data/EC_V4.xlsx', ec_id=1)
data.parse()

In [4]:
# Define generators - notice that in this case we only have 1 generator

gens = Generator(name='gens',
                 value=np.zeros(gen_resampled.shape),
                 lower_bound=np.zeros(gen_resampled.shape),
                 upper_bound=gen_resampled,
                 cost=data.generator['cost_parameter_b'][0, :],
                 cost_nde=data.generator['cost_nde'][0, :],
                 is_renewable=np.reshape(data.generator['type_generator'][0], (1, 1)))

In [5]:
# Define loads - same as above, only 1 load

loads = Load(name='loads',
             value=load_resampled,
             lower_bound=np.zeros(load_resampled.shape),
             upper_bound=load_resampled,
             cost=np.ones(load_resampled.shape),
             cost_cut=data.load['cost_cut'][0, :],
             cost_reduce=data.load['cost_reduce'][0, :],
             cost_ens=data.load['cost_ens'][0, :])

In [6]:
# Define storage

stor = Storage(name='stor',
               value=np.zeros(data.storage['p_charge_limit'].shape),
               lower_bound=np.zeros(data.storage['p_charge_limit'].shape),
               upper_bound=(data.storage['energy_capacity'] *
                            np.ones(data.storage['p_charge_limit'].shape).transpose()).transpose(),
               cost=np.ones(data.storage['p_charge_limit'].shape),
               cost_discharge=data.storage['discharge_price'],
               cost_charge=data.storage['charge_price'],
               capacity_max=data.storage['energy_capacity'],
               capacity_min=data.storage['energy_min_percentage'],
               initial_charge=data.storage['initial_state'],
               discharge_efficiency=data.storage['discharge_efficiency'],
               discharge_max=data.storage['p_discharge_limit'],
               charge_efficiency=data.storage['charge_efficiency'],
               charge_max=data.storage['p_charge_limit'],
               capital_cost=np.array([0.05250, 0.10500, 0.01575]))

In [7]:
# Define vehicle to grid

v2g = Vehicle(name='evs',
              value=np.zeros(data.vehicle['schedule_charge'].shape),
              lower_bound=(data.vehicle['e_capacity_max'] * data.vehicle['min_technical_soc'] *
                           np.ones(data.vehicle['schedule_charge'].shape).transpose()).transpose(),
              upper_bound=(data.vehicle['e_capacity_max'] * np.ones(data.vehicle['schedule_charge'].shape).transpose()).transpose(),
              cost=np.ones(data.vehicle['schedule_charge'].shape),
              cost_discharge=data.vehicle['discharge_price'][:, 0],
              cost_charge=data.vehicle['charge_price'][:, 0],
              capacity_max=data.vehicle['e_capacity_max'],
              initial_charge=np.ones(data.vehicle['schedule_charge'].shape) * 0.8,
              min_charge=data.vehicle['min_technical_soc'],
              discharge_efficiency=data.vehicle['discharge_efficiency'],
              charge_efficiency=data.vehicle['charge_efficiency'],
              capital_cost=np.array([0.042, 0.063, 0.042, 0.042, 0.063]),
              schedule_discharge=data.vehicle['schedule_discharge'],
              schedule_charge=data.vehicle['schedule_charge'])

In [8]:
# Define imports and exports

pimp = BinaryResource(name='pImp',
                      value=np.zeros(data.peers['import_contracted_p_max'][0, :].shape),
                      lower_bound=np.zeros(data.peers['import_contracted_p_max'][0, :].shape),
                      upper_bound=data.peers['import_contracted_p_max'][0, :],
                      cost=data.peers['buy_price'][0, :],
                      is_active=np.zeros(data.peers['import_contracted_p_max'][0, :].shape))

pexp = BinaryResource(name='pExp',
                      value=np.zeros(data.peers['export_contracted_p_max'][0, :].shape),
                      lower_bound=np.zeros(data.peers['export_contracted_p_max'][0, :].shape),
                      upper_bound=data.peers['export_contracted_p_max'][0, :],
                      cost=data.peers['sell_price'][0, :],
                      is_active=np.zeros(data.peers['export_contracted_p_max'][0, :].shape))

In [9]:
ec_components = {'gen': gens,
                 'loads': loads,
                 'stor': stor,
                 'evs': v2g,
                 'pimp': pimp,
                 'pexp': pexp}

In [10]:
# Create the scene

scene = HMProblemScene(name='EC1', data=ec_components, hm_parser=data)
scene.initialize()
scene.run()

100%|██████████| 200/200 [00:04<00:00, 44.74it/s]


In [11]:
# Population

scene.algo.population

array([[2.95039635e+03, 4.24825342e+01, 1.02672836e+02, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.49300109e+01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.89162146e+02, 1.42628422e+03, 1.61317866e+01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [2.04970420e+03, 0.00000000e+00, 1.22974059e+02, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [2.59408655e+03, 5.56791690e+02, 9.07635512e+01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [2.62153366e+03, 0.00000000e+00, 9.77587716e+01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00]])